## 简单的 RAG 

In [1]:
# !uv pip install -qU langchain-openai langchain-chroma langchain-text-splitters langchain-community langgraph langchain_ollama

In [2]:
import requests
import numpy as np

from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

EMBEDDING_MODEL_PATH = '../model/BAAI/bge-m3'

USER_AGENT environment variable not set, consider setting it to identify your requests.


在运行本 notebook 前，请先用 vllm 启动 qwen3 大模型推理服务。

**1）qwen3 模型推理服务**

大模型推理服务的相关配置:

- 虚拟环境部署教程在 `/test_qwen3/qwen3_vllm.ipynb`
- 模型下载脚本在 `/model/download_qwen.py`
- 启动脚本在 `/test_qwen3/vllm_server.sh`

下载模型并配置好虚拟环境后，来到项目根目录，打开 `test_qwen3` 文件夹，运行服务端启动脚本：

```bash
cd test_qwen3
bash vllm_server.sh
```

对于遵循 OpenAI 接口格式的本地模型服务，使用 `ChatOpenAI` 来连接。参考文档：[api.python.langchain.com](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)

In [3]:
llm = ChatOpenAI(
    model_name='Qwen3-0.6B-FP8',
    openai_api_base='http://localhost:8000/v1',
    openai_api_key='token-kcgyrk'
)

**2）bge-m3 文本 embedding 推理服务**

使用 [BAAI/bge-m3](https://www.modelscope.cn/models/BAAI/bge-m3) 作为文本 embedding 模型。

可以使用 Ollama 部署 bge-m3 文本 embedding 推理服务。安装和使用 [Ollama](https://ollama.com/) 的详细方法，参阅我的博客 [《本地部署大模型：Ollama 和 vLLM》](https://luochang212.github.io/posts/llm_deploy/)，这里就不详述了。

在 Windows 系统下，使用以下代码下载模型并启动推理服务：

```bash
# 查看 Ollama 版本
ollama --version

# 下载 bge-m3 模型
ollama pull bge-m3

# Windows Powershell 下，启动 Ollama 服务，指定端口为 11435
$env:OLLAMA_HOST="http://0.0.0.0:11435"; ollama serve
```

由于我的 `jupyter lab` 是在 Windows 的 wsl 的 Ubuntu 系统中运行的，因此我还需要获取 Windows 系统主机的 ip，以访问运行于主机上的 Ollama 服务。

在 Powershell 执行 `ipconfig` 命令，获取 IP 配置列表：

```
无线局域网适配器 WLAN:

   IPv4 地址 . . . . . . . . . . . . : 192.168.16.225

```

我的主机的 Ipv4 地址为 `192.168.16.225`，因此我的 `base_url="http://192.168.16.225:11435"`，你需要在这里替换成你的 IP。


In [4]:
base_url="http://192.168.16.225:11435"

In [5]:
# 使用 request 确认可以访问 Ollama 服务
response = requests.get(base_url, timeout=5)

print(f'response.status_code: {response.status_code}')
print(f'response.text: {response.text}')

response.status_code: 200
response.text: Ollama is running


In [6]:
embeddings = OllamaEmbeddings(
    model="bge-m3",
    base_url=base_url
)

texts = [
    "这是第一个示例文本",
    "这是第二个完全不同的文本",
    "这是第三个与第一个相似的文本"
]

# 使用 embed_documents 获取多个嵌入向量
embedding_vectors = embeddings.embed_documents(texts)

In [7]:
print(f'len(embedding_vectors): {len(embedding_vectors)}')
print(f'type(embedding_vectors): {type(embedding_vectors)}')
print(f'np.array(embedding_vectors).shape: {np.array(embedding_vectors).shape}')

len(embedding_vectors): 3
type(embedding_vectors): <class 'list'>
np.array(embedding_vectors).shape: (3, 1024)


In [8]:
# 使用 embed_query 获取单个查询嵌入
query = "这是一个测试文本"
query_embedding = embeddings.embed_query(query)
len(query_embedding)

1024

**3）使用向量数据库 Chroma**

In [9]:
vector_store = Chroma(
    collection_name="rag_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [10]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://luochang212.github.io/posts/docker_command/",)
)
docs = loader.load()

In [11]:
print(docs[0].metadata.keys())

dict_keys(['source', 'title', 'language'])


In [12]:
# print(docs[0].page_content)

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

10

In [14]:
# all_splits

In [15]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [16]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")
# prompt

/home/canva/miniconda3/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [21]:
response = graph.invoke({"question": "如何启动容器"})
print(response["answer"])

<think>
好的，用户的问题是如何启动容器。我需要从提供的上下文中找到相关的信息。首先，看到的Context中提到了在命令行中运行容器的步骤，包括使用docker run来启动。然后，还有两个具体的例子，分别是测试容器和运行容器。比如，测试容器使用了--rm、--gpus和--name等参数，而运行容器使用了-d、-p、-v等选项。

用户的问题只需要启动容器的方法，所以应该总结这两个例子中的关键步骤。要注意使用三个句子，确保简洁。同时，要确保答案准确，不偏离提供的上下文内容。检查是否有足够的信息，如果有的话，就直接引用。如果没有的话，可能需要判断是否还需要补充，但根据上下文，答案应该是两个命令的例子，所以用简洁的方式表达即可。
</think>

启动容器可通过以下步骤实现：  
1. 使用 `docker run` 命令指定镜像并运行容器，例如 `docker run --gpus all --name jupyter_server_app jupyter_server_image /bin/bash`。  
2. 或使用 `docker run` 的 `-d` 参数以后台模式运行，例如 `docker run -d -p 9999:8888 jupyter_server_image`，并结合端口映射和挂载路径进行配置。


参考：

- [Build a Retrieval Augmented Generation (RAG) App: Part 1](https://python.langchain.com/docs/tutorials/rag/)
- [Build a Retrieval Augmented Generation (RAG) App: Part 2](https://python.langchain.com/docs/tutorials/qa_chat_history/)